In [4]:


#CONNECT TO SPOTIFY

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

cid ="a09d11aa9cb341c899d56d84c67f5d7f" 
secret = "fa09d00c868d497ebbb160476b56f097"

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [3]:
#GET SONG NAMES OF RANDOM SAMPLE

# create empty lists where the results are going to be stored
artist_name = []
track_name = []
popularity = []
track_id = []

#random sample. set sample size, sample rate max 100, and song year
for i in range(0,100,10):
    track_results = sp.search(q='year:1990', type='track', limit=10,offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        popularity.append(t['popularity'])

        
print('number of elements in the track_id list:', len(track_id))


#imports track names
import pandas as pd
df_tracks = pd.DataFrame({'artist_name':artist_name,'track_name':track_name,'track_id':track_id,'popularity':popularity})
print(df_tracks.shape)
df_tracks.head()



number of elements in the track_id list: 100
(100, 4)


,artist_name,track_name,track_id,popularity
0,AC/DC,Thunderstruck,57bgtoPSgt236HzfBOd8kj,78
1,Elton John,Don't Go Breaking My Heart,5pKJtX4wBeby9qIfFhyOJj,69
2,Alice In Chains,Man in the Box,6gZVQvQZOFpzIy3HblJ20F,68
3,Bell Biv DeVoe,Poison,6m59VvDUi0UQsB2eZ9wVbH,63
4,A Tribe Called Quest,Can I Kick It?,3Ti0GdlrotgwsAVBBugv0I,68


In [50]:
#SONG NAME DATA CLEANUP

grouped = df_tracks.groupby(['artist_name','track_name'], as_index=True).size()
grouped[grouped > 1].count()

df_tracks.drop_duplicates(subset=['artist_name','track_name'], inplace=True)

# doing the same grouping as before to verify the solution
grouped_after_dropping = df_tracks.groupby(['artist_name','track_name'], as_index=True).size()
grouped_after_dropping[grouped_after_dropping > 1].count()

df_tracks[df_tracks.duplicated(subset=['artist_name','track_name'],keep=False)].count()
df_tracks.shape

(96, 4)

In [51]:
#GET SONG METRICS OF THE RANDOM SAMPLE


# empty list, batchsize and the counter for None results
rows = []
batchsize = 50
None_counter = 0


for i in range(0,len(df_tracks['track_id']),batchsize):
    batch = df_tracks['track_id'][i:i+batchsize]
    feature_results = sp.audio_features(batch)
    for i, t in enumerate(feature_results):
        if t == None:
            None_counter = None_counter + 1
        else:
            rows.append(t)
            
            
print('Number of tracks where no audio features were available:',None_counter)
print('number of elements in the track_id list:', len(rows))


df_audio_features = pd.DataFrame.from_dict(rows,orient='columns')
print("Shape of the dataset:", df_audio_features.shape)
df_audio_features.head()
df_audio_features.info()

Number of tracks where no audio features were available: 0
number of elements in the track_id list: 96
Shape of the dataset: (96, 18)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 18 columns):
acousticness        96 non-null float64
analysis_url        96 non-null object
danceability        96 non-null float64
duration_ms         96 non-null int64
energy              96 non-null float64
id                  96 non-null object
instrumentalness    96 non-null float64
key                 96 non-null int64
liveness            96 non-null float64
loudness            96 non-null float64
mode                96 non-null int64
speechiness         96 non-null float64
tempo               96 non-null float64
time_signature      96 non-null int64
track_href          96 non-null object
type                96 non-null object
uri                 96 non-null object
valence             96 non-null float64
dtypes: float64(9), int64(4), object(5)
memory usage: 13

In [52]:
#SONG METRICS DATA CLEANUP

columns_to_drop = ['analysis_url','track_href','type','uri']
df_audio_features.drop(columns_to_drop, axis=1,inplace=True)

df_audio_features.rename(columns={'id': 'track_id'}, inplace=True)

df_audio_features.shape

(96, 14)

In [53]:
# MERGE BOTH DATAFRAMES

# the 'inner' method will make sure that we only keep track IDs present in both datasets
df = pd.merge(df_tracks,df_audio_features,on='track_id',how='inner')
print("Shape of the dataset:", df_audio_features.shape)
df.head()
df.info()

df[df.duplicated(subset=['artist_name','track_name'],keep=False)]

Shape of the dataset: (96, 14)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 96 entries, 0 to 95
Data columns (total 17 columns):
artist_name         96 non-null object
track_name          96 non-null object
track_id            96 non-null object
popularity          96 non-null int64
acousticness        96 non-null float64
danceability        96 non-null float64
duration_ms         96 non-null int64
energy              96 non-null float64
instrumentalness    96 non-null float64
key                 96 non-null int64
liveness            96 non-null float64
loudness            96 non-null float64
mode                96 non-null int64
speechiness         96 non-null float64
tempo               96 non-null float64
time_signature      96 non-null int64
valence             96 non-null float64
dtypes: float64(9), int64(5), object(3)
memory usage: 13.5+ KB


,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence


In [54]:
#WRITE TO CSV

df.to_csv('try1.csv')